In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, interact_manual, GridBox, Layout, VBox
import plotly.graph_objs as go

%matplotlib widget

In [ ]:
FEATURES = ["Forest", "Pasture", "Urban"]

### Functions that call a model

In [ ]:
def run_prescriptor(context):
    return dict(zip(FEATURES, context))


def run_predictor(context, actions):
    sum = 0
    for c in context:
        sum += c
    for a in actions:
        sum += a

    return sum

### Demo

In [ ]:
fig = go.FigureWidget()

"""
Submits context and creates pie chart
Updates sliders for pie chart accordingly
#TODO: handle case where percentages don't sum to 1
"""
def prescribe(b):
    data = [box.value for box in boxes.values()]
    if sum(data) == 100:
        prescribed = run_prescriptor(data)

        for feature in FEATURES:
            sliders[feature].value = prescribed[feature]

        # Clear figure and re-plot
        fig.data = []
        fig.add_pie(values=data, labels = FEATURES)

    else:
        with(prescribe_output):
            print("Please make sure your percentage sums to 100")


"""
Locks a slider so it isn't affected by the sum to 100 computation
"""
def lock(change):
    if change["new"]:
        locked.add(change["owner"])
    else:
        locked.remove(change["owner"])


"""
Real-time updater for pie chart
# TODO: this breaks when a value hits 0
"""
def update(change):
    with fig.batch_update():
        if len(fig["data"]) > 0:

            owner = change["owner"]

            # First compute what percentage is locked
            locked_sum = 0
            for feat in sliders:
                if sliders[feat] != owner and ticks[feat] in locked:
                    locked_sum += sliders[feat].value

            # Add locked percentage to old and new because we don't factor
            # them in to the 100% in our calculating the new value
            old = change["old"] + locked_sum
            new = change["new"] + locked_sum

            # Adjust each slider that isn't being currently modified
            for feat in sliders:
                slider = sliders[feat]
                tick = ticks[feat]
                if slider != owner and tick not in locked:
                    # Unobserve so we don't infinitely recurse
                    slider.unobserve(update, names="value")
                    # old value / old total = new value / new total
                    slider.value = slider.value / (100 - old) * (100 - new)
                    slider.observe(update, names="value")

            # Update pie chart
            fig["data"][0]["values"] = [slider.value for slider in sliders.values()]


"""
Submits context and actions and outputs prediction
"""
def predict(b):
    with predict_output:
        context = [box.value for box in boxes.values()]
        actions = [slider.value for slider in sliders.values()]
        outcome = run_predictor(context, actions)
        print("ELUC: ", outcome)
        

"""
Construct widgets and attach them to their functions
TODO: handle unusable land
"""
boxes = {feature : widgets.FloatText(value=0.0, description=feature) for feature in FEATURES}
sliders = {feature : widgets.FloatSlider(value=0.0, description=feature + "_diff") for feature in FEATURES}
ticks = {feature : widgets.Checkbox(value=False, description="lock_" + feature) for feature in FEATURES}

# For use in locking and unlocking sliders
locked = set()

prescribe_button = widgets.Button(description="Prescribe")
prescribe_output = widgets.Output()
prescribe_button.on_click(prescribe)


predict_button = widgets.Button(description="Predict")
predict_output = widgets.Output()
predict_button.on_click(predict)


"""
Display Interactables and Figures
TODO: add titles, make layout prettier
"""
for box in boxes.values():
    display(box)

display(prescribe_button, prescribe_output)

display(fig)

for feat in sliders:
    sliders[feat].observe(update, names="value")
    ticks[feat].observe(lock, names="value")
    display(sliders[feat], ticks[feat])

display(predict_button, predict_output)